UFC DATA SCAPER

Auther: Phillip Simmonds

In [94]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as req
import datetime
import uuid

FIRST: COLLECT THE EVNTS DATA

In [42]:
events_list_url = "http://ufcstats.com/statistics/events/completed?page=all"

In [43]:
events_dictionary = {
    "eventurl" : [],
    "eventurlid" : [],
    "datestring" : [],
    "fighttitlefullname" : [],
    "fighttitlecategory" : [],
    "location" : []
    }

In [44]:
def create_dict_for_events(fight_event):
    event_url = str(fight_event.find("a")).split('"')[3]
    event_url_id = event_url.split("/")[-1]
    date_string = fight_event.find("span").text.replace("\n","").strip()
    fight_title_full_name = fight_event.find("a").text.replace("\n","").strip()
    fight_title_category = fight_title_full_name.split(":")[0]
    location = fight_event.find_all("td")[-1].text.replace("\n","").strip()
    events_dictionary["eventurl"].append(event_url)
    events_dictionary["eventurlid"].append(event_url_id)
    events_dictionary["datestring"].append(date_string)
    events_dictionary["fighttitlefullname"].append(fight_title_full_name)
    events_dictionary["fighttitlecategory"].append(fight_title_category)
    events_dictionary["location"].append(location)


In [45]:
events = BeautifulSoup(req.get(events_list_url).content, 'html.parser').find("table","b-statistics__table-events")
for fight_event in events.find_all("tr","b-statistics__table-row")[2:]:
    create_dict_for_events(fight_event)


In [46]:
event_df = pd.DataFrame.from_dict(events_dictionary)
event_df.to_csv("extraction/events.csv")

SECOND: COLLECT THE FIGHT NIGHT DATA

In [47]:
def sp(txt):
    return txt.text.strip()


In [48]:
fighter_items = []

In [51]:

for i,row in event_df[["eventurl","eventurlid"]].iterrows():
   fighters_table = BeautifulSoup(req.get(row.eventurl).content, 'html.parser').find("tbody")
   fights = fighters_table.find_all("tr")
   for fight in fights:

      fight_id = str(uuid.uuid4()) # Unique id for the fight 
      fighter_one_winner = [fight_id, row.eventurlid]
      fighter_two = [fight_id, row.eventurlid]
      col_vals = fight.find_all("td")

      if "win" in col_vals[0].text:
         fighter_one_winner.append("WIN")
         fighter_two.append("LOSS")
      else:
         fighter_one_winner.append("NC")
         fighter_two.append("NC")
      
      names = col_vals[1].find_all("p")
      # Name 
      fighter_one_winner.append(sp(names[0]))
      fighter_two.append(sp(names[1]))

      
      fighter_one_url = str(names[0].find("a")).split('"')[-2]
      fighter_two_url = str(names[1].find("a")).split('"')[-2]
      fighter_one_id = fighter_one_url.split("/")[-1]
      fighter_two_id = fighter_two_url.split("/")[-1]

      # Fighter id
      fighter_one_winner.append(fighter_one_id)
      fighter_two.append(fighter_two_id)

      fighter_one_winner.append(fighter_one_url)
      fighter_two.append(fighter_two_url)

      # Fight stats
      knock_downs = col_vals[2].find_all("p")
      strikes = col_vals[3].find_all("p")
      take_downs = col_vals[4].find_all("p")
      submissions = col_vals[5].find_all("p")

      fighter_one_winner.append(sp(knock_downs[0]))
      fighter_two.append(sp(knock_downs[1]))
      fighter_one_winner.append(sp(strikes[0]))
      fighter_two.append(sp(strikes[1]))
      fighter_one_winner.append(sp(take_downs[0]))
      fighter_two.append(sp(take_downs[1]))
      fighter_one_winner.append(sp(submissions[0]))
      fighter_two.append(sp(submissions[1]))

      # Class and Bonus
      class_and_bonus = col_vals[6].find("p")
      w_class = sp(class_and_bonus)
      if len(class_and_bonus.find_all("img")) == 1:
         bonus = True
      else:
         bonus = False

      fighter_one_winner.append(w_class)
      fighter_two.append(w_class)
      fighter_one_winner.append(bonus)
      fighter_two.append(False)

      method = sp(col_vals[7].find_all("p")[0])
      method_2 = sp(col_vals[7].find_all("p")[1])
      fighter_one_winner.append(method)
      fighter_two.append(method)
      fighter_one_winner.append(method_2)
      fighter_two.append(method_2)


      round_end = sp(col_vals[8].find("p"))
      fighter_one_winner.append(round_end)
      fighter_two.append(round_end)

      final_round_time = sp(col_vals[9].find("p"))
      fighter_one_winner.append(final_round_time)
      fighter_two.append(final_round_time)

      fighter_items.append(fighter_one_winner)
      fighter_items.append(fighter_two)

    

In [54]:
header_ = [
"fightid",
"eventurlid",
"result",
"fullname",
"fighterid",
"fighterdetailsurl",
"knockdowns",
"strikes",
"takedowns",
"submission_attempts",
"weightclass",
"fightbonus",
"win_method",
"methodaction",
"finalround",
"finalround_finshtime"
]

In [55]:
fighter_df = pd.DataFrame(data=fighter_items, columns = header_)
fighter_df

,fightid,eventurlid,result,fullname,fighterid,fighterdetailsurl,knockdowns,strikes,takedowns,submission_attempts,weightclass,fightbonus,win_method,methodaction,finalround,finalround_finshtime
0,437ed9d8-b5fe-4b0c-a273-1eaee71507cd,1fac46d466abd5b8,WIN,Curtis Blaydes,fa6796c55d6c5440,http://ufcstats.com/fighter-details/fa6796c55d...,1,30,0,0,Heavyweight,True,KO/TKO,Punch,2,0:17
1,437ed9d8-b5fe-4b0c-a273-1eaee71507cd,1fac46d466abd5b8,LOSS,Chris Daukaus,d0aaedcf7ccd0b45,http://ufcstats.com/fighter-details/d0aaedcf7c...,0,13,0,0,Heavyweight,False,KO/TKO,Punch,2,0:17
2,df07cc3a-aafa-438b-961a-c0d6980e65ae,1fac46d466abd5b8,WIN,Alexa Grasso,e8b731feff72294b,http://ufcstats.com/fighter-details/e8b731feff...,0,16,2,1,Women's Flyweight,False,SUB,Rear Naked Choke,1,3:57
3,df07cc3a-aafa-438b-961a-c0d6980e65ae,1fac46d466abd5b8,LOSS,Joanne Wood,12f91bfa8f1f723b,http://ufcstats.com/fighter-details/12f91bfa8f...,0,34,0,0,Women's Flyweight,False,SUB,Rear Naked Choke,1,3:57
4,a293da22-527d-4a13-884f-d140c3122b6b,1fac46d466abd5b8,WIN,Bryan Barberena,a331233f597090a5,http://ufcstats.com/fighter-details/a331233f59...,0,111,0,0,Welterweight,True,S-DEC,,3,5:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,11682cad-c13b-4668-bc38-3e5ce863b55a,a6a9ab5a824e8f66,LOSS,David Levicki,49590e0508b2c19f,http://ufcstats.com/fighter-details/49590e0508...,0,4,0,0,Open Weight,False,KO/TKO,Punches,1,12:13
13092,3928116c-2d62-4223-b6ab-7db9a6e643e9,a6a9ab5a824e8f66,WIN,Patrick Smith,46c8ec317aff28ac,http://ufcstats.com/fighter-details/46c8ec317a...,0,1,0,1,Open Weight,False,SUB,Guillotine Choke,1,0:58
13093,3928116c-2d62-4223-b6ab-7db9a6e643e9,a6a9ab5a824e8f66,LOSS,Ray Wizard,ea0ad155451ed1f5,http://ufcstats.com/fighter-details/ea0ad15545...,0,1,0,0,Open Weight,False,SUB,Guillotine Choke,1,0:58
13094,510530e4-4e1b-4b64-9d99-81a88961afdb,a6a9ab5a824e8f66,WIN,Scott Morris,be9d259be012e8a4,http://ufcstats.com/fighter-details/be9d259be0...,0,1,1,1,Open Weight,False,SUB,Guillotine Choke,1,0:20


In [56]:
fighter_df.to_csv("extraction/fight_records.csv")

NEXT STEP IS GET THE FIGHTER DETAILS

In [114]:
def cstats(txt):
    return txt.text.replace("\n","").split(":")[1].strip()


fighter_ids = fighter_df[["fullname","fighterid","fighterdetailsurl"]].drop_duplicates("fighterid")
lastupdate = datetime.datetime.now()
fighter_details = []
for i,row in fighter_ids.iterrows():
    fighter_stats = BeautifulSoup(req.get(row.fighterdetailsurl).content, 'html.parser').find_all("div", "b-list__info-box")
    info = fighter_stats[0].find_all("li")
    stats = fighter_stats[1].find("div", "b-list__info-box-left").find_all("li")
    fighter_details.append([
            row["fighterid"],
            row["fullname"],
            cstats(info[0]),
            cstats(info[1]),
            cstats(info[2]),
            cstats(info[3]),
            cstats(info[4]),
            cstats(stats[0]),
            cstats(stats[1]),
            cstats(stats[2]),
            cstats(stats[3]),
            cstats(stats[5]),
            cstats(stats[6]),
            cstats(stats[7]),
            cstats(stats[8]),
            lastupdate])
    
    # request the above url to get fighter information
    # then we can create a players dimmension 

In [115]:
cols = [
    "id",
    "fullname",
    "Height",
    "Weight",
    "Reach",
    "STANCE",
    "DOB",
    "SLpM",
    "StrAcc",
    "SApM",
    "StrDef",
    "TDAvg",
    "TDAcc",
    "TDDef",
    "SubAvg",
    "lastupdated"
]
df_details = pd.DataFrame(data = fighter_details, columns = cols)
df_details

,id,fullname,Height,Weight,Reach,STANCE,DOB,SLpM,StrAcc,SApM,StrDef,TDAvg,TDAcc,TDDef,SubAvg,lastupdated
0,fa6796c55d6c5440,Curtis Blaydes,"6' 4""",265 lbs.,"80""",Orthodox,"Feb 18, 1991",3.52,51%,1.68,60%,6.06,53%,33%,0.0,2022-03-31 21:46:11.970931
1,d0aaedcf7ccd0b45,Chris Daukaus,"6' 3""",250 lbs.,"76""",Orthodox,"Sep 25, 1989",6.47,50%,4.02,59%,0.00,0%,100%,0.0,2022-03-31 21:46:11.970931
2,e8b731feff72294b,Alexa Grasso,"5' 5""",125 lbs.,"66""",Orthodox,"Aug 09, 1993",4.92,43%,3.79,63%,0.54,57%,60%,0.7,2022-03-31 21:46:11.970931
3,12f91bfa8f1f723b,Joanne Wood,"5' 6""",125 lbs.,"65""",Orthodox,"Dec 23, 1985",6.76,50%,4.54,52%,1.53,55%,59%,0.4,2022-03-31 21:46:11.970931
4,a331233f597090a5,Bryan Barberena,"6' 0""",170 lbs.,"72""",Southpaw,"May 03, 1989",5.61,49%,4.83,45%,0.17,25%,55%,0.3,2022-03-31 21:46:11.970931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,c3c23c99477c041b,Frank Hamaker,--,--,--,,--,0.00,0%,0.00,0%,0.00,0%,0%,0.0,2022-03-31 21:46:11.970931
2249,505934897b8b4824,Thaddeus Luster,"6' 3""",210 lbs.,--,,--,0.00,0%,0.00,0%,0.00,0%,0%,0.0,2022-03-31 21:46:11.970931
2250,49590e0508b2c19f,David Levicki,"6' 5""",275 lbs.,--,,--,0.00,0%,0.00,0%,0.00,0%,0%,0.0,2022-03-31 21:46:11.970931
2251,ea0ad155451ed1f5,Ray Wizard,--,--,--,,--,0.00,0%,0.00,0%,0.00,0%,0%,0.0,2022-03-31 21:46:11.970931


In [139]:
# Add additional data
ft_ = df_details.Height
cm = []
for ft in ft_:
    if "'" in ft:
        h_inch = float(ft.split("'")[1].replace('"',"").strip())
        h_inch += float(ft.split("'")[0]) * 12
        h_cm = round(h_inch * 2.54, 2)
        cm.append(h_cm)
    else:
        cm.append("")


wt_ = df_details.Weight
mass = []
for we in wt_:
    if "lb" in we:
        kg = round(float(we.split()[0]) * 0.453592, 2)
        mass.append(kg)
    else:
        mass.append("")


rc_ = df_details.Reach
reach = []
for rc in rc_:
    if '"' in rc:
        r_cm = round(float(rc.split('"')[0]) * 2.54, 2)
        reach.append(r_cm)
    else:
        reach.append("")

df_details["HightCM"] = cm
df_details["WeightKG"] = mass
df_details["ReachCM"] = reach


In [140]:
df_details.to_csv("extraction/fighter_details.csv")